In [28]:
import numpy as np

fname = 'assign1_data.csv'
data = np.loadtxt(fname, dtype='float', delimiter=',', skiprows=1)
X, y = data[:, :-1], data[:, -1].astype(int)
X_train, y_train = X[:400], y[:400]
X_test, y_test = X[400:], y[400:]

In [29]:
class DenseLayer:

    def __init__(self, n_inputs, n_neurons, seed):
        np.random.seed(seed)
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(self.inputs, self.weights) + self.biases

    def backward(self, dz):
        self.dweights = np.dot(self.inputs.T, dz)
        self.dbiases = np.sum(dz, axis=0, keepdims=True)
        self.dinputs = np.dot(dz, self.weights.T)

In [30]:
class ReLu:

    def forward(self, z):
        self.z = z
        self.activity = np.maximum(0, self.z)

    def backward(self, dactivity):
        self.dz = dactivity.copy()
        self.dz[self.z <= 0] = 0.0

In [31]:
class Softmax:

    def forward(self, z):
        e_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        self.probs = e_z / e_z.sum(axis=1, keepdims=True)
        return self.probs

    def backward(self, dprobs):
        self.dz = np.empty_like(dprobs)
        for i, (prob, dprob) in enumerate(zip(self.probs, dprobs)):
            prob = prob.reshape(-1, 1)
            jacobian = np.diagflat(prob) - np.dot(prob, prob.T)
            self.dz[i] = np.dot(jacobian, dprob)

In [32]:
class CrossEntropyLoss:

    def forward(self, probs, oh_y_true):
        probs_clipped = np.clip(probs, 1e-7, 1 - 1e-7)
        loss = -np.sum(oh_y_true * np.log(probs_clipped), axis=1)
        return loss.mean(axis=0)

    def backward(self, probs, oh_y_true):
        batch_sz, n_class = probs.shape
        self.dprobs = -oh_y_true / probs
        self.dprobs = self.dprobs / batch_sz

In [33]:
class SGD:

    def __init__(self, learning_rate=1.0):
        self.learning_rate = learning_rate

    def update_params(self, layer):
        layer.weights = layer.weights - self.learning_rate * layer.dweights
        layer.biases = layer.biases - self.learning_rate * layer.dbiases

In [34]:
def predictions(probs):
    y_preds = np.argmax(probs, axis=1)
    return y_preds

In [35]:
def accuracy(y_preds, y_true):
    return np.mean(y_preds == y_true)

In [36]:
# A single forward pass through the entire network.
def forward_pass(X, y_true, oh_y_true):
    dense1.forward(X)
    activation1.forward(dense1.z)
    dense2.forward(activation1.activity)
    activation2.forward(dense2.z)
    dense3.forward(activation2.activity)
    probs = output_activation.forward(dense3.z)
    loss = crossentropy.forward(probs, oh_y_true)
    return probs, loss

In [37]:
# A single backward pass through the entire network.
def backward_pass(probs, y_true, oh_y_true):
    crossentropy.backward(probs, oh_y_true)
    output_activation.backward(crossentropy.dprobs)
    dense3.backward(output_activation.dz)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dz)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dz)

In [38]:
# Initialize the network and set hyperparameters
epochs = 10
n_class = 3
n_batch = 20
dense1 = DenseLayer(3, 4, 1)
dense2 = DenseLayer(4, 8, 2)
dense3 = DenseLayer(8, 3, 3)
activation1 = ReLu()
activation2 = ReLu()
output_activation = Softmax()
crossentropy = CrossEntropyLoss()
optimizer = SGD()

In [39]:
# Training loop
for epoch in range(epochs):
    print('epoch:', epoch)
    for batch_i in range(n_batch):
        x = np.split(X_train, n_batch)[batch_i]
        y_true = np.split(y_train, n_batch)[batch_i]
        oh_y_true = np.eye(n_class)[y_true]
        forward_pass(x, y_true, oh_y_true)
        probs, loss = forward_pass(x, y_true, oh_y_true)
        y_preds = predictions(probs)
        print('Accuracy: ', accuracy(y_preds, y_true), 'Loss: ', loss)
        backward_pass(probs, y_true, oh_y_true)
        optimizer.update_params(dense3)
        optimizer.update_params(dense2)
        optimizer.update_params(dense1)

epoch: 0
Accuracy:  0.3 Loss:  1.0986093183882306
Accuracy:  0.3 Loss:  1.103065175859254
Accuracy:  0.4 Loss:  1.0916882286067806
Accuracy:  0.35 Loss:  1.0965938478155728
Accuracy:  0.25 Loss:  1.100965864399321
Accuracy:  0.2 Loss:  1.111827326912512
Accuracy:  0.5 Loss:  1.06129740627571
Accuracy:  0.25 Loss:  1.1740542352924332
Accuracy:  0.3 Loss:  1.1145310704972966
Accuracy:  0.1 Loss:  1.1041176663006924
Accuracy:  0.25 Loss:  1.1729584787466316
Accuracy:  0.5 Loss:  1.1103139591683093
Accuracy:  0.2 Loss:  1.1603536287880685
Accuracy:  0.4 Loss:  1.0797167402611336
Accuracy:  0.15 Loss:  1.180627199662128
Accuracy:  0.35 Loss:  1.114466236515785
Accuracy:  0.6 Loss:  1.092478013491839
Accuracy:  0.25 Loss:  1.1740225413636778
Accuracy:  0.25 Loss:  1.0867239302269396
Accuracy:  0.35 Loss:  1.117433097325489
epoch: 1
Accuracy:  0.3 Loss:  1.1219871983402954
Accuracy:  0.3 Loss:  1.1054150731735284
Accuracy:  0.4 Loss:  1.0997270012013936
Accuracy:  0.35 Loss:  1.09731573571026

In [40]:
test_probs, test_loss = forward_pass(X_test, y_test, np.eye(n_class)[y_test])
test_y_preds = predictions(test_probs)
print(accuracy(test_y_preds, y_test))

0.95
